In [1]:
import sys
import copy 
import random
import itertools
import math
import pandas as pd
import numpy as np
import torch
import tqdm.auto as tqdm
from pathlib import Path
import unittest

from typing import *
from enum import Enum
from dataclasses import dataclass

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("research")

In [4]:
import os,sys
sys.path.insert(0, str(Path.cwd().parent))

In [5]:
import utils.logging
utils.logging.setup(debug=False)

### We're playing **Narde** variation here, as it was the one we played in my childhood.
And these are the [rules](https://www.bkgm.com/variants/Narde.html)

In [6]:
from backgammon.game import *

## Player interface & automation

In [7]:
iters = 100
samples = 1000
means = []
for i in tqdm.trange(iters, leave=False):
    means.append(sum([sum(AutoGame().dice) for _ in tqdm.trange(samples, leave=False)]) / samples)
means = np.array(means)
print(f"avg dice sum is {means.mean():.2f}, and lays within range ({means.mean() - 3*means.std():.2f} +/- {means.mean() + 3*means.std():.2f}) with 3 sigmas")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

avg dice sum is 8.18, and lays within range (7.77 +/- 8.58) with 3 sigmas


In [8]:
summarize(AutoGame(player1=LazyPlayer(), player2=LazyPlayer()))

Result(winner=1, turns=80, reward=1)

In [9]:
simulate(AutoGame(player2=LazyPlayer()), player1=LazyPlayer())

Result(winner=0, turns=94, reward=-1)

In [10]:
simulate(AutoGame(), player1=LazyPlayer(), player2=LazyPlayer())

Result(winner=1, turns=92, reward=1)

In [19]:
import math

games = 1000

results = []
experiments = list(itertools.product(["lazy", "rand"], ["lazy", "rand"], ["random", "first", "second"]))
random.shuffle(experiments)
for p1, p2, start in tqdm.tqdm(experiments, leave=False):
    exp_info = {"p1": p1, "p2": p2, "start": start}
    player1 = LazyPlayer() if p1 == "lazy" else RandomPlayer()
    player2 = LazyPlayer() if p2 == "lazy" else RandomPlayer()
    start_args = [6,6] if start == "random" else ([6,1] if start == "first" else [1,6])
    sims = pd.DataFrame([summarize(AutoGame(player1, player2, False).start(*start_args)).__dict__ for _ in tqdm.trange(games, postfix=exp_info, leave=False)])
    exp_info["games"] = games
    exp_info["wins"] = sims["winner"].sum()
    wins_mu = exp_info["wins"] / exp_info["games"]
    wins_sd = round(math.sqrt(exp_info["games"] * wins_mu * (1 - wins_mu)), 2)
    exp_info["win_rate_lo"] = ((exp_info["wins"] - wins_sd*3) / exp_info["games"]).round(2)
    exp_info["win_rate_mu"] = wins_mu
    exp_info["win_rate_hi"] = ((exp_info["wins"] + wins_sd*3) / exp_info["games"]).round(2)
    exp_info["avg_turns"] = sims["turns"].mean()
    exp_info["avg_reward"] = sims["reward"].mean()
    results.append(exp_info)

pd.DataFrame(results).sort_values(by=["p1", "p2", "start"])

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s, p1=lazy, p2=rand, start=first]

  0%|          | 0/1000 [00:00<?, ?it/s, p1=lazy, p2=lazy, start=second]

  0%|          | 0/1000 [00:00<?, ?it/s, p1=rand, p2=lazy, start=first]

  0%|          | 0/1000 [00:00<?, ?it/s, p1=rand, p2=rand, start=second]

  0%|          | 0/1000 [00:00<?, ?it/s, p1=rand, p2=lazy, start=random]

  0%|          | 0/1000 [00:00<?, ?it/s, p1=rand, p2=rand, start=first]

  0%|          | 0/1000 [00:00<?, ?it/s, p1=rand, p2=lazy, start=second]

  0%|          | 0/1000 [00:00<?, ?it/s, p1=lazy, p2=lazy, start=random]

Interesting bias is observed here, as lazy policy seems to have a significant advantage when played against random.  
Bias only shows up if it plays as player1, but disappears if it plays as player2

TODO: Figure out what's going on here